In [0]:
# Databricks notebook source
# ================================================================
# 02_download_gdrive - Baixar XMLs do Google Drive
# ================================================================
# Pré-requisito: instalar dependências no cluster:
# %pip install pydrive2 lxml

In [0]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os

In [0]:
# ===== Configurações =====
SERVICE_ACCOUNT_JSON = "/dbfs/FileStore/service_account.json"  # credenciais Google
FOLDER_ID = "1Uf-wYgbi5ZDsqB-mF6zI5ZP9QmM6xqub"                # ID da pasta GDrive
LOCAL_TMP = "/tmp/gdrive_downloads"
DBFS_PATH = "dbfs:/mnt/amazonia/raw/xml/"

os.makedirs(LOCAL_TMP, exist_ok=True)

In [0]:
# ===== Autenticação =====
gauth = GoogleAuth()
gauth.LoadCredentialsFile(SERVICE_ACCOUNT_JSON)
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()

drive = GoogleDrive(gauth)

In [0]:
# ===== Listar arquivos XML na pasta =====
file_list = drive.ListFile({'q': f"'{FOLDER_ID}' in parents and trashed=false"}).GetList()

for f in file_list:
    if f['mimeType'] == 'application/xml':
        filename = os.path.join(LOCAL_TMP, f['title'])
        f.GetContentFile(filename)
        print(f"Baixado: {filename}")

        # Copiar para DBFS
        dbutils.fs.cp(f"file://{filename}", DBFS_PATH + f['title'])